In [1]:
import numpy as np
import cv2 
%matplotlib inline
from matplotlib import pyplot as plt
from scipy import ndimage
from numpy import linalg as LA
import math as m

In [67]:
cap = cv2.VideoCapture("videos/test.mp4")
angle = []

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")
while cap.isOpened():
    
    ret, frame = cap.read()
    if frame is None:
        break
        
    frame = cv2.resize(frame,(720,720))
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    

    #Binary thresholding
    
    binary_img_red = cv2.inRange(hsv_frame, (160, 100, 100), (180, 255, 255))
    binary_img_yellow = cv2.inRange(hsv_frame, (30, 100, 100), (70, 255, 255))
    binary_img_blue = cv2.inRange(hsv_frame, (70, 100, 100), (140, 255, 255))
    
    result = binary_img_red + binary_img_yellow + binary_img_blue
    
    # Vectorizing
    point=np.zeros((720,720,3),np.uint8)
    xblue,yblue = ndimage.measurements.center_of_mass(binary_img_blue)
    xred,yred = ndimage.measurements.center_of_mass(binary_img_red)
    xyellow,yyellow= (xred+20,300)
   
    # Cas Limites
    if m.isnan(xblue) or m.isnan(yblue):
        xblue = 0
        yblue=0
    if m.isnan(xred) or m.isnan(yred):
        xred =0 
        yred = 0
    if m.isnan(yyellow) or m.isnan(xyellow):
        yyellow = 0
        xyellow =0
    
    # Points correspondants aux post-its
    pointr = cv2.circle(point, (int(yred),int(xred)), radius=10, color=(0, 0, 255), thickness=-1)
    pointb = cv2.circle(point, (int(yblue),int(xblue)), radius=10, color=(0, 0, 50), thickness=-1)
    pointy = cv2.circle(point, (int(yyellow),int(xyellow)), radius=10, color=(0, 0, 150), thickness=-1)
    
    point_result = pointr + pointb + pointy
    
    # Vecteurs du bras
    stationary = np.array([xyellow-xred, yyellow-yred])
    moving = np.array([xblue-xred, yblue-yred])
    
    #Calcul de l'angle
    angle.append(np.dot(stationary,moving)/ (LA.norm(stationary) * LA.norm(moving)))
    cv2.imshow("Vectors", point_result)
    
    #Pour arreter le traitement et sauvarder les images
    if cv2.waitKey(1) == ord('q'):
        cv2.imwrite("videos/Frame1.jpeg", frame)
        cv2.imwrite("videos/Hsv_Frame.jpeg", hsv_frame)
        cv2.imwrite("videos/result.jpeg", result)
        cv2.imwrite("videos/points.jpeg", point_result)
        break
        
        
cap.release()
cv2.destroyAllWindows()

In [18]:

angle = np.arccos(angle)*180/np.pi
np.save("real", angle)